In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio
from rasterio.enums import Resampling
import matplotlib.colors as mcolors
import xarray as xr
import cftime
import math
import rioxarray
from shapely.geometry import mapping
import cartopy.crs as ccrs
import matplotlib as mpl

In [25]:
from pathlib import Path
import sys
import os

path_mod = f"{Path(os.path.dirname(os.path.abspath(''))).parents[0]}/"
print(path_mod)
sys.path.append(path_mod)
from src.indicators.food_insecurity.config import Config
from src.utils_general.utils import download_ftp,download_url
from src.utils_general.raster_manipulation import fix_calendar, invert_latlon, change_longitude_range
from src.utils_general.plotting import plot_raster_boundaries_clip

/Users/tinkavalentijn/Documents/CHD/AAFI/pa-anticipatory-action/


#### Set config values

In [33]:
country="malawi"
admin_level=2
suffix=""
config=Config()
parameters = config.parameters(country)
country_dir = os.path.join(config.DIR_PATH, "analyses", country)
country_data_raw_dir = os.path.join(config.DATA_DIR, 'raw', country)
country_data_processed_dir = os.path.join(config.DATA_DIR, 'processed', country)
country_data_exploration_dir = os.path.join(config.DATA_DIR,"exploration",country)
drought_data_exploration_dir= os.path.join(config.DATA_DIR, "exploration",  'drought')
cams_data_dir=os.path.join(drought_data_exploration_dir,"CAMS_OPI")
cams_tercile_path=os.path.join(cams_data_dir,"CAMS_tercile.nc")
chirps_monthly_dir=os.path.join(drought_data_exploration_dir,"CHIRPS")
chirps_monthly_path=os.path.join(chirps_monthly_dir,"chirps_global_monthly.nc")

In [34]:
adm1_bound_path=os.path.join(country_data_raw_dir,config.SHAPEFILE_DIR,parameters["path_admin1_shp"])
adm2_bound_path=os.path.join(country_data_raw_dir,config.SHAPEFILE_DIR,parameters["path_admin2_shp"])

In [35]:
fewsnet_dir = os.path.join(country_data_processed_dir, config.FEWSWORLDPOP_PROCESSED_DIR)
fewsnet_filename = config.FEWSWORLDPOP_PROCESSED_FILENAME.format(country=country,admin_level=admin_level,suffix=suffix)

In [36]:
globalipc_dir=os.path.join(country_data_processed_dir, config.GLOBALIPC_PROCESSED_DIR)
globalipc_path=os.path.join(globalipc_dir,f"{country}_globalipc_admin{admin_level}{suffix}.csv")

In [60]:
df_fn=pd.read_csv(os.path.join(fewsnet_dir,fewsnet_filename),index_col=False)
df_fn["source"]="FewsNet"

In [61]:
df_fn.head()

,ADMIN0,ADMIN1,ADMIN2,date,CS_1,CS_2,CS_99,ML1_1,ML1_2,ML1_99,...,ML2_2m,perc_ML2_2m,perc_inc_ML2_3p,perc_inc_ML1_3p,CS_88,ML1_88,ML2_88,period_ML1,period_ML2,source
0,Malawi,Central,Dedza,2009-07-01,663581.06250,0.000000,2496.515625,663581.06250,0.000000,2496.515625,...,663581.06250,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
1,Malawi,Southern,Zomba City,2009-07-01,79436.00000,0.000000,4183.390625,79436.00000,0.000000,4183.390625,...,79436.00000,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
2,Malawi,Southern,Zomba,2009-07-01,527573.87500,57.163952,98510.125000,527573.87500,57.163952,98510.125000,...,527573.87500,99.989166,0.010834,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
3,Malawi,Southern,Thyolo,2009-07-01,642996.12500,0.000000,0.000000,642996.12500,0.000000,0.000000,...,642996.12500,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
4,Malawi,Southern,Phalombe,2009-07-01,336895.46875,0.000000,1237.324707,336895.46875,0.000000,1237.324707,...,336895.46875,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet


In [62]:
df_gipc=pd.read_csv(globalipc_path)
df_gipc["source"]="GlobalIPC"

In [63]:
df_gipc.head()

,date,period_ML1,period_ML2,ADMIN0,ADMIN1,ADMIN2,CS_1,CS_2,CS_3,CS_4,...,perc_ML1_4p,perc_ML1_2m,perc_ML2_4p,perc_ML2_2m,perc_inc_ML2_3p,perc_inc_ML1_3p,pop_CS,pop_ML1,pop_ML2,source
0,2017-07-01,Oct 2017 - Mar 2018,NaN,malawi,NaN,Balaka,260933.0,71851.0,34035.0,0.0,...,0.0,78.000021,NaN,NaN,NaN,12.721561,366819.0,378164.0,NaN,GlobalIPC
1,2017-07-01,Oct 2017 - Mar 2018,NaN,malawi,NaN,Blantyre,255879.0,134032.0,0.0,0.0,...,0.0,82.999924,NaN,NaN,NaN,17.000076,389911.0,406157.0,NaN,GlobalIPC
2,2017-07-01,Oct 2017 - Mar 2018,NaN,malawi,NaN,Chikwawa,418157.0,111509.0,11151.0,0.0,...,0.0,79.000023,NaN,NaN,NaN,18.938096,540817.0,557453.0,NaN,GlobalIPC
3,2017-07-01,Oct 2017 - Mar 2018,NaN,malawi,NaN,Chiradzulu,258449.0,45801.0,13086.0,0.0,...,0.0,88.000037,NaN,NaN,NaN,7.876258,317336.0,327151.0,NaN,GlobalIPC
4,2017-07-01,Oct 2017 - Mar 2018,NaN,malawi,NaN,Chitipa,182978.0,20793.0,0.0,0.0,...,0.0,100.000000,NaN,NaN,NaN,0.000000,203771.0,187136.0,NaN,GlobalIPC


In [68]:
df_ipc=pd.concat([df_fn,df_gipc])

In [69]:
df_ipc.head()

,ADMIN0,ADMIN1,ADMIN2,date,CS_1,CS_2,CS_99,ML1_1,ML1_2,ML1_99,...,ML2_2m,perc_ML2_2m,perc_inc_ML2_3p,perc_inc_ML1_3p,CS_88,ML1_88,ML2_88,period_ML1,period_ML2,source
0,Malawi,Central,Dedza,2009-07-01,663581.06250,0.000000,2496.515625,663581.06250,0.000000,2496.515625,...,663581.06250,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
1,Malawi,Southern,Zomba City,2009-07-01,79436.00000,0.000000,4183.390625,79436.00000,0.000000,4183.390625,...,79436.00000,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
2,Malawi,Southern,Zomba,2009-07-01,527573.87500,57.163952,98510.125000,527573.87500,57.163952,98510.125000,...,527573.87500,99.989166,0.010834,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
3,Malawi,Southern,Thyolo,2009-07-01,642996.12500,0.000000,0.000000,642996.12500,0.000000,0.000000,...,642996.12500,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
4,Malawi,Southern,Phalombe,2009-07-01,336895.46875,0.000000,1237.324707,336895.46875,0.000000,1237.324707,...,336895.46875,100.000000,0.000000,0.0,NaN,NaN,NaN,Jul - Sep 2009,Oct - Dec 2009,FewsNet
